# The Company K Case Study

As part of the Supply Chain Management and Logistics Network Design course - Master's degree in Management Engineering - University of Bergamo

(c) Prof. Roberto Pinto

**This notebook contains all the data and functions required by the Company K case study. Press shift + enter in each cell to run the commands, or click on Run in the command bar.**

In [ ]:
#RIMUOVERE PRIMA DEL RILASCIO
%load_ext autoreload
%autoreload 2

In [ ]:
# LASCIARE SOLO NELLA VERSIONE COLAB
# Prepare the notebook
!git clone --branch projectwork_24_25 https://github.com/ropinotex/network_optimization.git
import sys
sys.path.insert(0,'/content/network_optimization')

!pip install -r /content/network_optimization/requirements.txt

Import functions and data

In [ ]:
# Functions
from netopt_ui import netopt_ui, edit_warehouse_ui, add_warehouse_ui, delete_warehouse_ui
from netopt_utils import plot_map, print_solution, show_assignments
from netopt_compat import netopt
from data_structures import (show_data, show_geo_map, set_capacity, set_all_capacities, set_fixed_cost,
                             set_all_fixed_costs, scale_demand, scale_all_demands, set_demand,
                             set_all_demands, get_demand, get_capacity)
# Data
from data.projectwork24_25_data import warehouses, customers

The functions available are the following:
- netopt_ui() - solver's user interface to solve the optimization model
- add_warehouse_ui() - user interface to add a new warehouse
- edit_warehouse_ui() - user interface to edit warehouses's data
- delete_warehouse_ui() - user interface to delete a warehouse

Other low level functions are available to work with the data programmatically:
- netopt() - solver function (without user interface)
- set_capacity() - changes the capacity of a warehouse
- set_all_capacities() - changes the capacity of all warehouses
- set_fixed_cost() - changes the yearly fixed cost of a warehouse
- set_all_fixed_costs() - changes the yearly fixed cost of all warehouses
- scale_demand() - scales the demand of a customer by a factor
- scale_all_demands() - scales the demand of all customer by a factor
- set_demand() - changes the demand of a customer
- set_all_demands() - changes the demand of all customers
- print_solution() - prints detail of the solution
- plot_map() - plots the map of customers and warehouses
- show_geo_map() - plots the map of customers and warehouses on a geo graphic map
- show_assignments() - displays the assignements
- show_data() - shows the data in tabular form


**Type the function name without parentheses and preceeded by _?_ to access a short description of the function and the parameters**

Example:

In [ ]:
?netopt_ui

**Show data**

You can explore the data in several ways. Using show_data() provides a convenient way to display customers and warehouses. 
The capacity of the warehouses represents the maximum throughput, whereas the customers demand represent the estimated orders from each customer.

In [ ]:
show_data(customers)
# or
# show_data(warehouses)

You can simply show the variable content


In [ ]:
customers

**Plot the data**

You can control the colors and shapes in the plot using the following parameters (also in the netopt function):
- warehouse_marker=shape of the warehouse icons. Allowed values are s=square, o=circle, *=star, ^=triangle, v=inverted triangle. Default is s 
- warehouse_markercolor=color of the warehouse icons. Allowed values are red, green, blue, black, yellow. Default is red
- warehouse_markersize=size of the warehouse icons. Default is 4
- warehouse_active_markersize=size of the warehouse icons representing active (open) warehouses. Default is 5
- customer_marker=shape of the customer icons. Default is o
- customer_markercolor=color of the customer icons. Default is blue
- customer_markersize=size of the customer icons. Default is 4

If you don't specify the above parameters about the figures, the default values will be assumed

In [ ]:
plot_map(customers=customers,
         warehouses=warehouses,
         warehouse_marker='s',
         warehouse_markercolor='red',
         warehouse_markersize=6,
         customer_marker='o',
         customer_markercolor='blue',
         customer_markersize=3)

A better map can be shown using the following command. However, this map allows only for the
visualization of location. To show the connections in the final solution you should use the plot_map command

In [ ]:
show_geo_map(warehouses=warehouses, customers=customers, zoom=6)

**Run the user interface**

The user interface allows you control the solver. The solver uses the data passed as arguments as _warehouses_ and _customers_.

- **Problem type**: The type of problem to be solved
- **# Warehouses**: Number of warehouses to activate (value of _p_ in the p-median problems)
- **Func. to minimize**: Minimize either the average waighted distance (AWD) or the total cost (only for p-median problems)
- **Service radius**: Defines the catchment area in a p-cover problem
- **Dist. Ranges**: Defines the cut points to measure the % of demand within given distances (i.e. [0, 100, 400] measures the % of demand within 100, 400, and beyond 400 km from any active warehouse)
- **Force single sourcing**: If active, each customer can be served by only on active warehouse
- **Force uncapacitated**: If active, neglects the capacities of the warehouse (only for p-median problems)
- **Ignore fixed cost**: If active. neglects the warehouses' fixed cost
- **Force open**: Ensures that the listed warehouses are active in the final solution
- **Force closed**: Ensures that the listed warehouses are NOT active in the final solution
- **Force allocations**: Ensures that a given customer is allocated to a given warehouse (if the warehouse is active in the final solution) 
- **Mutually exclusive**: Prevents two warehouses to be simultaneously active at the same time in the final solution
- **Unit transport cost**: Transportation cost per unit and km

**Warning:** The solver has a time limit of 120 seconds. The solution is within a 5% of real optimum. 

**Always check your data (warehouses and customers) before running the solver to be sure you are actually solving the right problem!**

In [ ]:
netopt_ui(warehouses=warehouses, customers=customers)

If you prefer, you can run the same solver programmatically without using the user interface

In [ ]:
results = netopt(
        num_warehouses=9,
        warehouses=warehouses,
        customers=customers,
        objective="p-median",
        objective_function="mincost",
        unit_transport_cost= 0.025,
        mutually_exclusive=[],
        plot=True,
        plot_size= (8, 12),
        hide_inactive=False,
        force_single_sourcing=False,
        force_uncapacitated=False,
        ignore_fixed_cost=False,
        force_open=[],
        force_closed=[],
        force_allocations=[],
        warehouse_marker="s",
        warehouse_markercolor="red",
        warehouse_markersize=6,
        customer_marker="s",
        customer_markercolor="blue",
        customer_markersize=4,
        print_model=True
    )

To show the assignments, run the following with the result of the _netopt()_ function: 

In [ ]:
show_assignments(results)

**Service levels**

By adding the parameter distance_ranges the functions returns the % of the demand within the passed distance ranges. 
For example, if distance_ranges = [0, 100, 400] the functions return the percentage of demand in the ranges [0, 100], (100, 400], (400, 99999]
where 99999 is used to represent a very long distance (i.e. infinite distance).

The parameter distance_ranges must be a list of increasing numbers. If you do not pass 0 as the first value it will be automatically added


**Force warehouses open or close**

It is possible to force warehouses to be open (for example, to force using the current warehouse) or closed (to avoid the selection of some candidates).

The warehouses are references through their id, and must be passed as list [] (even for a single value, that is to force closed the warehouse with id 1 you should pass force_closed=[1])

The id of the warehouses can be found in the 'warehouses' variable using the *show_data()* method.


**Mutually exclusive facilities**

In some cases, some facilities may be _mutually exclusive_, that is the presence of one facility excludes the activation of another one and viceversa.

For example, let's assume that facilities 1, 9 and 15 are mutually exclusive (either one of them or none of them can be selected). Similarly, warehouse 2 and 4 are mutually exclusive (but they are not exclusive with respect to 1, 9 and 15). This can be formulated by passing the parameter

_mutually_exclusive = [(1, 9, 15), (2, 4)]_

It is possible to set any number of mutually exclusive sets (each set is a tuple in a list). This constraint is useful when we have different alternatives (for example, different sizes) for the same facility.

**Remove the single-source constraint**

The model implicitly stipulates the single-source constraint (each customer is served by exactly one warehouse). When dealing with capacities, it may be useful to relax this constraint to better exploit the available capacity.



## Change the data ##

There are different ways to change the data. You can use _add_warehouse_ui()_, _edit_warehouse_ui()_ and _delete_warehouse_ui()_ to change the warehouses data using a user interface. You must pass the variable containing the data as parameter.

In [ ]:
add_warehouse_ui(warehouses=warehouses)

In [ ]:
# Pay attention: you should pass both the variable containing the warehouse data and the id of the warehouse to modify
edit_warehouse_ui(warehouses=warehouses, warehouse_id=5)

In [ ]:
delete_warehouse_ui(warehouses=warehouses)

There are also other function to programmatically change data for both warehouses and customers:

**Warehouses**

- **set_capacity**: Change the capacity of a warehouse. The models consider capacity as throughput of the warehouse.
- **set_all_capacities**: Change the capacity of all warehouses with the given capacity
- **set_fixed_cost**: Change the yearly fixed_cost of the warehouse.
- **set_all_fixed_costs**: Change the fixed_cost of all warehouses with the given fixed_cost

**Customers**

- **scale_demand**: Scale a customer's demand by factor (i.e. multiplies the current customer's demand by a factor provided by the user)
- **scale_all_demands**: Scale all customers' demand by factor
- **set_demand**: Set a customer's demand to a given value
- **set_all_demands**: Set all customers' demand to a given value


## From here on, it is up to you! ##